# Regression LSTM with best parameters
    find the best prediction window to apply w/ lr = 0.0001

# Import libraries

In [4]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import pandas as pd;
import numpy as np;
import sklearn
import matplotlib.pyplot as plt
import time

date =  [2018010000, 
         2018030000, 
         2018050000,
         2018070000, 
         2018090000, 
         2018110000]

# parameters
steps = 96
n_hidden = 2
units = 150
batch_size = 336
epochs = 100
features_num = 14

# lists to append results
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
y_pred_list = []
time_count = []

# Import keras libraries, packages and data:

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers

# import data
data_full = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# Create loop for different dates:

In [6]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# design the LSTM
def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 1:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

  
# LOOP STARTS
for i in date:
    start_time = time.time()
    # data
    data = data_full.loc[data_full.index > i, :]

    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)

    # fill nan values in the whole data set
    data.fillna(data.mean(), inplace = True)

    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False)  
    
    from sklearn.preprocessing import MinMaxScaler

    # data scaling  (including offer (y))
    sc_X = MinMaxScaler()
    data_train = sc_X.fit_transform(data_train)
    data_test = sc_X.transform(data_test)
    
    # divide features and labels
    X_train = data_train[:, 0:14] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:14] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    model = regressor_tunning()
    
    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = epochs,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15 (inverso should not be used as scalling was made with the whole data set)
    y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
    y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)
    
    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    # Need to process data with spike occurences the same way as features
    data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

    # set predictive window according with tuning best results
    data = data.loc[data.index > i, :]

    # make sure shaded area will correspond to values outputed by LSTM
    data.reset_index(drop = True, inplace = True)

    # fill_nan is already made - so lets split data into test and train
    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    shade_train, shade_test = train_test_split(
             data, test_size = 0.15, shuffle = False)

    # reset index of testing data
    shade_test.reset_index(drop = True, inplace = True)

    # function to split data into correct shape for RNN
    def split_data(shade_test, steps):
        y_spike_occ = list()
        upper_lim = list()
        lower_lim = list()
        for i in range(steps, len(shade_test.index)):
            y_spike_occ.append(shade_test['spike_occurance'][i])
            upper_lim.append(shade_test['spike_upperlim'][i])
            lower_lim.append(shade_test['spike_lowerlim'][i])
        return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

    # function to cut data set so it can be divisible by the batch_size
    def cut_data(data, batch_size):
         # see if it is divisivel
        condition = data.shape[0] % batch_size
        if condition == 0:
            return data
        else:
            return data[: -condition]
    
    # shape y_spike_occ for the right size to compare results in normal and spike regions
    y_spike_occ, spike_upperlim, spike_lowerlim = split_data(shade_test, steps)
    y_spike_occ = cut_data(y_spike_occ, batch_size)

    # continue
    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)
    
    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/100
37/37 [==============================] - 53s 1s/step - loss: 0.3685 - mse: 0.3685 - mae: 0.3985 - val_loss: 0.0561 - val_mse: 0.0561 - val_mae: 0.2350
Epoch 2/100
37/37 [==============================] - 51s 1s/step - loss: 0.1164 - mse: 0.1164 - mae: 0.2717 - val_loss: 0.0461 - val_mse: 0.0461 - val_mae: 0.2132
Epoch 3/100
37/37 [==============================] - 56s 2s/step - loss: 0.0623 - mse: 0.0623 - mae: 0.2011 - val_loss: 0.0057 - val_mse: 0.0057 - val_mae: 0.0711
Epoch 4/100
37/37 [==============================] - 54s 1s/step - loss: 0.0371 - mse: 0.0371 - mae: 0.1539 - val_loss: 0.0163 - val_mse: 0.0163 - val_mae: 0.1257
Epoch 5/100
37/37 [==============================] - 58s 2s/step - loss: 0.0193 - mse: 0.0193 - mae: 0.1090 - val_loss: 9.3821e-04 - val_mse: 9.3821e-04 - val_mae: 0.0265
Epoch 6/100
37/37 [==============================] - 52s 1s/step - loss: 0.0119 - mse: 0.0119 - mae: 0.0857 - val_loss: 0.0109 - val_mse: 0.0109 - val_mae: 0.1018
Epoch 7/100
37

Epoch 49/100
37/37 [==============================] - 54s 1s/step - loss: 9.8261e-04 - mse: 9.8261e-04 - mae: 0.0179 - val_loss: 5.5802e-04 - val_mse: 5.5802e-04 - val_mae: 0.0187
Epoch 50/100
37/37 [==============================] - 61s 2s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0200 - val_loss: 4.9979e-04 - val_mse: 4.9979e-04 - val_mae: 0.0161
Epoch 51/100
37/37 [==============================] - 68s 2s/step - loss: 9.8474e-04 - mse: 9.8474e-04 - mae: 0.0184 - val_loss: 5.1739e-04 - val_mse: 5.1739e-04 - val_mae: 0.0168
Epoch 52/100
37/37 [==============================] - 57s 2s/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0194 - val_loss: 4.9075e-04 - val_mse: 4.9075e-04 - val_mae: 0.0157
Epoch 53/100
37/37 [==============================] - 56s 2s/step - loss: 9.1849e-04 - mse: 9.1849e-04 - mae: 0.0175 - val_loss: 5.6269e-04 - val_mse: 5.6269e-04 - val_mae: 0.0187
Epoch 54/100
37/37 [==============================] - 59s 2s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0203 - val_l

37/37 [==============================] - 70s 2s/step - loss: 8.7315e-04 - mse: 8.7315e-04 - mae: 0.0170 - val_loss: 4.6805e-04 - val_mse: 4.6805e-04 - val_mae: 0.0157
Epoch 96/100
37/37 [==============================] - 65s 2s/step - loss: 8.5059e-04 - mse: 8.5059e-04 - mae: 0.0169 - val_loss: 4.4684e-04 - val_mse: 4.4684e-04 - val_mae: 0.0149
Epoch 97/100
37/37 [==============================] - 65s 2s/step - loss: 8.6993e-04 - mse: 8.6993e-04 - mae: 0.0170 - val_loss: 4.3309e-04 - val_mse: 4.3309e-04 - val_mae: 0.0141
Epoch 98/100
37/37 [==============================] - 55s 1s/step - loss: 8.3257e-04 - mse: 8.3257e-04 - mae: 0.0169 - val_loss: 4.3846e-04 - val_mse: 4.3846e-04 - val_mae: 0.0146
Epoch 99/100
37/37 [==============================] - 60s 2s/step - loss: 8.8323e-04 - mse: 8.8323e-04 - mae: 0.0174 - val_loss: 4.2582e-04 - val_mse: 4.2582e-04 - val_mae: 0.0138
Epoch 100/100
37/37 [==============================] - 67s 2s/step - loss: 9.1242e-04 - mse: 9.1242e-04 - mae: 0.

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


TypeError: split_data() takes 2 positional arguments but 3 were given

In [ ]:
results = pd.DataFrame({                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})

y_pred = pd.DataFrame({'dates': date,
                       'Predicitons': y_pred_list})

y_pred.to_csv('Pedictions_LSTM_5_prediction_window.csv')

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
results.min()

In [ ]:
!pip install matplotlib

import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

dates_labels = ['12 ',
                '10 ',
                '8 ',
                '6 ',
                '4 ',
                '2 ']

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged RMSE for different\n predictive windows')
plt.plot(rmse_gen, label = 'Overall error')
plt.plot(rmse_spi, label = 'Spike regions')
plt.plot(rmse_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('RMSE_predictive_window.png')

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged MAE for different\n predictive windows')
plt.plot(mae_gen, label = 'Overall error')
plt.plot(mae_spi, label = 'Spike regions')
plt.plot(mae_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('MAE_predictive_window.png')